# Global Charts

## Datasets
- [Global Economic Monitor (GEM)](https://databank.worldbank.org/reports.aspx?source=1179)

- [GDP per capita, PPP (constant 2021 international $)](https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.KD)

- [Real effective exchange rate index (2010 = 100)](https://data.worldbank.org/indicator/PX.REX.REER)

## Preamble

In [ ]:
import pandas as pd
import wbdata
import plotly.express as px
import plotly.graph_objects as go

import parser

## World Bank Organisation Data

In [ ]:
cpi_df = await parser.parse_worldbank_dataset("data/datasets/global_cpi.json")
gdp_df = await parser.parse_worldbank_dataset("data/datasets/global_gdp.json")
reer2010_df = await parser.parse_worldbank_dataset("data/datasets/global_reer2010.json")

In [ ]:
df_raw = cpi_df.merge(gdp_df, on=["country", "date"]).merge(reer2010_df, on=["country", "date"])
df_raw

In [ ]:
country_code_map: dict[str, str] = {
    entry["name"]: entry["id"]
    for entry in wbdata.get_countries()
}

In [ ]:
country_filter = [
    "United Kingdom",
    "Morocco",
    "South Africa",
    "Singapore",
    "Ireland",
    "Malaysia",
    "Japan",
    "China",
    "United States",
    "Brazil",
    "Chile",
]

In [ ]:
df = df_raw.copy()
df = df.dropna()
df = df[df["date"] >= 2014]
df = df.rename({
    "country": "Country",
    "date": "Year",
    "CPTOTSAXN": "CPI",
    "NY.GDP.PCAP.PP.KD": "PPP",
    "PX.REX.REER": "REER2010",
}, axis=1)

df = df.reset_index(drop=True)

df = df[df["Country"].isin(country_filter)]

countries_with_all_years = df.groupby('Country')['Year'].apply(lambda years: set(range(2014, 2024)).issubset(set(years)))
valid_countries = countries_with_all_years[countries_with_all_years].index

df = df[df['Country'].isin(valid_countries)]

df["Country Code"] = df["Country"].apply(lambda name: country_code_map[name])
df = df[["Year", "Country", "Country Code", "CPI", "PPP", "REER2010"]] # Reorder columns.

df["CPI Text"] = df["CPI"].apply(round)
df["PPP Text"] = df["PPP"].apply(round)
df["CPI PPP Text"] = df.apply(
    lambda row: f"<b>CPI:</b> {row["CPI Text"]}, <b>PPP:</b> {row["PPP Text"]}" if row["Country"] == "Singapore" else "",
    axis=1
)

df = df.sort_values(["Year", "Country"])
df

## CPI and GDP Map

In [ ]:
fig = px.scatter_geo(
    df,
    locations="Country Code",
    size="CPI",
    text="CPI Text",
    color="Country",
    hover_name="Country",
    size_max=30,
    animation_frame="Year",
    title="Bubble Map of CPI by Country",
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=800,
)

# Need to increase the resolution of the map.
# Otherwise, Singapore won't be visible.
fig.update_layout({
    "geo": {
        "resolution": 50,
        "projection": {
            "type": "natural earth",
        },
    },
})

fig.update_traces(
    textposition="middle left",
    textfont=dict(weight="bold", color="black"),
)

# Change text position for specific countries for better visibility.
fig.update_traces(
    selector=dict(name="United Kingdom"),
    textposition="middle right",
)

fig.update_traces(
    selector=dict(name="Japan"),
    textposition="middle right",
)

fig.update_traces(
    selector=dict(name="Malaysia"),
    textposition="middle right",
)

# Border for Singapore.
fig.update_traces(
    selector=dict(name='Singapore'),
    marker=dict(line=dict(width=2, color='black'))
)

fig.show()

In [ ]:
fig = px.scatter_geo(
    df,
    locations="Country Code",
    size="PPP",
    text="PPP Text",
    color="Country",
    hover_name="Country",
    size_max=30,
    animation_frame="Year",
    title="Bubble Map of GDP per Capita (PPP) by Country",
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=800,
)

# Need to increase the resolution of the map.
# Otherwise, Singapore won't be visible.
fig.update_layout({
    "geo": {
        "resolution": 50,
        "projection": {
            "type": "natural earth",
        },
    },
})

fig.update_traces(
    textposition="middle left",
    textfont=dict(weight="bold", color="black"),
)

# Change text position for specific countries for better visibility.
fig.update_traces(
    selector=dict(name="United Kingdom"),
    textposition="middle right",
)

fig.update_traces(
    selector=dict(name="Japan"),
    textposition="middle right",
)

fig.update_traces(
    selector=dict(name="Malaysia"),
    textposition="middle right",
)

# Border for Singapore.
fig.update_traces(
    selector=dict(name='Singapore'),
    marker=dict(line=dict(width=2, color='black'))
)

fig.show()

## CPI vs. GDP per Capita (PPP) with REER Bubble Size

In [ ]:
fig = px.scatter(
    df,
    x="CPI",
    y="PPP",
    size="REER2010",
    color="Country",
    text="CPI PPP Text",
    hover_name="Country",
    hover_data=["Year"],
    animation_frame="Year",
    title="CPI vs. GDP per Capita (PPP) with Real Effective Exchange Rate (2010=100) Bubble Size",
    labels={"CPI": "Consumer Price Index (CPI)", "PPP": "GDP per Capita (PPP)", "REER2010": "Real Effective Exchange Rate (2010=100)"},
    range_x=[50, 250],
    range_y=[0, 150_000],
    width=800
)

# Border for all points.
fig.update_traces(marker=dict(
    line=dict(width=1, color='black')
))

# Set text position.
fig.update_traces(
    textposition="middle right",
    textfont=dict(color="black"),
)

# Thicker border for Singapore.
fig.update_traces(
    selector=dict(name="Singapore"),
    marker=dict(line=dict(width=3, color='black'))
)
        
fig.show()